In [155]:
import warnings
warnings.simplefilter('ignore')
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import confusion_matrix,classification_report,roc_auc_score,accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
import numpy as np

df=pd.read_csv("C:\\Users\\Neel\\Desktop\\Greyatom Codes\\Datasets\\Social_Network_Ads.csv")
df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [156]:
df["Purchased"].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [157]:
df["Gender"]=df["Gender"].astype("category")
df["Gender"]=df["Gender"].cat.codes

### Logistic Regression without handling class imbalance

In [158]:
X=df.drop(["User ID","Purchased"],1)
y=df["Purchased"]

In [159]:
scaler=MinMaxScaler()

In [160]:
X=pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [161]:
X.head()

,Gender,Age,EstimatedSalary
0,1.0,0.023810,0.029630
1,1.0,0.404762,0.037037
2,0.0,0.190476,0.207407
3,0.0,0.214286,0.311111
4,1.0,0.023810,0.451852


In [162]:
X_train,X_test,y_train,y_test=tts(X,y,test_size=0.3,random_state=42)

In [163]:
logreg_model=LogisticRegression()

In [164]:
logreg_model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [165]:
y_pred=logreg_model.predict(X_test)

In [166]:
accuracy_score(y_test,y_pred)

0.85

In [167]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.81      0.99      0.89        73
           1       0.97      0.64      0.77        47

    accuracy                           0.85       120
   macro avg       0.89      0.81      0.83       120
weighted avg       0.87      0.85      0.84       120



### Logistic Regression using SMOTE

In [168]:
from imblearn.over_sampling import SMOTE

In [169]:
sm=SMOTE(random_state=42)

In [170]:
df=df.drop(["User ID"],1)

In [171]:
new_df=pd.DataFrame(scaler.fit_transform(df),columns=df.columns)

In [172]:
new_df.head()

,Gender,Age,EstimatedSalary,Purchased
0,1.0,0.023810,0.029630,0.0
1,1.0,0.404762,0.037037,0.0
2,0.0,0.190476,0.207407,0.0
3,0.0,0.214286,0.311111,0.0
4,1.0,0.023810,0.451852,0.0


In [173]:
X=new_df.drop(["Purchased"],1)
y=df["Purchased"]

In [174]:
X_res,y_res=sm.fit_sample(X,y.ravel())

In [175]:
X_train,X_test,y_train,y_test=tts(X_res,y_res,test_size=0.3,random_state=42)

In [176]:
logreg=LogisticRegression()

In [177]:
logreg.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [178]:
y_pred=logreg.predict(X_test)

In [179]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.80      0.84      0.82        75
           1       0.84      0.80      0.82        80

    accuracy                           0.82       155
   macro avg       0.82      0.82      0.82       155
weighted avg       0.82      0.82      0.82       155



### Logistic Regression Using GridSearchCV

In [180]:
credit=pd.read_csv("C:\\Users\\Neel\\Desktop\\Greyatom Codes\\Datasets\\creditcard.csv")
credit.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [181]:
credit.shape

(284807, 31)

In [182]:
credit["Class"].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [183]:
credit["Time"]= sscaler.fit_transform(np.array(credit["Time"]).reshape(-1,1))
credit["Amount"]= sscaler.fit_transform(np.array(credit["Amount"]).reshape(-1,1))

In [184]:
credit.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000
mean,-1.050379e-14,3.919560e-15,5.688174e-16,-8.769071e-15,2.782312e-15,-1.552563e-15,2.010663e-15,-1.694249e-15,-1.927028e-16,-3.137024e-15,...,1.537294e-16,7.959909e-16,5.367590e-16,4.458112e-15,1.453003e-15,1.699104e-15,-3.660161e-16,-1.206049e-16,3.202236e-16,0.001727
std,1.000002e+00,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,1.000002e+00,0.041527
min,-1.996583e+00,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,-3.532294e-01,0.000000
25%,-8.552120e-01,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,-3.308401e-01,0.000000
50%,-2.131453e-01,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,-2.652715e-01,0.000000
75%,9.372174e-01,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,-4.471707e-02,0.000000
max,1.642058e+00,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,1.023622e+02,1.000000


In [185]:
log_reg=LogisticRegression()

In [186]:
#Considering 1000 random samples of Class 0 and 10 random samples of Class 1. To reduce the time of GridSearchCV
zeros=credit.sample(n=1000,random_state=42)
ones=credit[credit["Class"]==1].sample(n=10,random_state=42)

In [187]:
credit=pd.concat([zeros,ones])

In [188]:
credit.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
43428,-1.122574,-16.526507,8.584972,-18.649853,9.505594,-13.793819,-2.832404,-16.701694,7.517344,-8.507059,...,1.190739,-1.127670,-2.358579,0.673461,-1.413700,-0.462762,-2.018575,-1.042804,1.102834,1
49906,-1.064538,0.339812,-2.743745,-0.134070,-1.385729,-1.451413,1.015887,-0.524379,0.224060,0.899746,...,-0.213436,-0.942525,-0.526819,-1.156992,0.311211,-0.746647,0.040996,0.102038,1.726255,0
29474,-1.249364,1.399590,-0.590701,0.168619,-1.029950,-0.539806,0.040444,-0.712567,0.002299,-0.971747,...,0.102398,0.168269,-0.166639,-0.810250,0.505083,-0.232340,0.011409,0.004634,-0.229289,0
276481,1.522680,-0.432071,1.647895,-1.669361,-0.349504,0.785785,-0.630647,0.276990,0.586025,-0.484715,...,0.358932,0.873663,-0.178642,-0.017171,-0.207392,-0.157756,-0.237386,0.001934,-0.347232,0
278846,1.551109,2.014160,-0.137394,-1.015839,0.327269,-0.182179,-0.956571,0.043241,-0.160746,0.363241,...,-0.238644,-0.616400,0.347045,0.061561,-0.360196,0.174730,-0.078043,-0.070571,-0.349671,0


In [189]:
X=credit.drop(["Class"],1)
y=credit["Class"]

In [190]:
X_train,X_val,y_train,y_val=tts(X,y,test_size=0.2,random_state=42)

In [191]:
X=X_train
y=y_train

In [192]:
params={"C":np.arange(0.001,10,0.2),"penalty":["l1","l2"]}

In [193]:
log_reg_cv=GridSearchCV(log_reg,param_grid=params,cv=5)

In [194]:
log_reg_cv.fit(X,y)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': array([1.000e-03...
       4.801e+00, 5.001e+00, 5.201e+00, 5.401e+00, 5.601e+00, 5.801e+00,
       6.001e+00, 6.201e+00, 6.401e+00, 6.601e+00, 6.801e+00, 7.001e+00,
       7.201e+00, 7.401e+00, 7.601e+00, 7.801e+00, 8.001e+00, 8.201e+00,
       8.401e+00, 8.601e+00, 8.801e+00, 9.001e+00, 9.201e+00, 9.401e+00,
 

In [195]:
log_reg_cv.best_params_

{'C': 0.201, 'penalty': 'l2'}

In [196]:
best_model=log_reg_cv.best_estimator_

In [197]:
y_pred=best_model.predict(X_val)

In [198]:
print(classification_report(y_val,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       201
           1       0.00      0.00      0.00         1

    accuracy                           1.00       202
   macro avg       0.50      0.50      0.50       202
weighted avg       0.99      1.00      0.99       202



### Applying SMOTE to same problem to remove class imbalance

In [199]:
credit=pd.read_csv("C:\\Users\\Neel\\Desktop\\Greyatom Codes\\Datasets\\creditcard.csv")
credit.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [200]:
sm=SMOTE(random_state=0)

In [201]:
credit["Time"]= sscaler.fit_transform(np.array(credit["Time"]).reshape(-1,1))
credit["Amount"]= sscaler.fit_transform(np.array(credit["Amount"]).reshape(-1,1))

In [202]:
zeros=credit.sample(n=1000,random_state=42)
ones=credit[credit["Class"]==1].sample(n=10,random_state=42)

In [203]:
short_credit=pd.concat([zeros,ones])

In [204]:
X=short_credit.drop(["Class"],1)
y=short_credit["Class"]

In [205]:
X_res,y_res=sm.fit_sample(X,y)

In [206]:
pd.Series(y_res).value_counts()

1    998
0    998
dtype: int64

In [207]:
logistic=LogisticRegression()
X_train, X_test, y_train, y_test = tts(X_res,y_res, random_state = 42, test_size = 0.3)

In [208]:
params = {'C' : np.arange(0.001,10,0.2), 'penalty' : ['l1','l2']}  
new_log_reg_cv=GridSearchCV(logistic,params,cv=5)

In [209]:
new_log_reg_cv.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': array([1.000e-03...
       4.801e+00, 5.001e+00, 5.201e+00, 5.401e+00, 5.601e+00, 5.801e+00,
       6.001e+00, 6.201e+00, 6.401e+00, 6.601e+00, 6.801e+00, 7.001e+00,
       7.201e+00, 7.401e+00, 7.601e+00, 7.801e+00, 8.001e+00, 8.201e+00,
       8.401e+00, 8.601e+00, 8.801e+00, 9.001e+00, 9.201e+00, 9.401e+00,
 

In [210]:
best_new_model=new_log_reg_cv.best_estimator_

In [211]:
y_pred=best_new_model.predict(X_test)

In [212]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       301
           1       0.99      1.00      1.00       298

    accuracy                           1.00       599
   macro avg       1.00      1.00      1.00       599
weighted avg       1.00      1.00      1.00       599



In [214]:
accuracy_score(y_test,y_pred)

0.996661101836394